# LSTMで体のモーションを推定する  
## モーションキャプチャ（OptiTrack + Motive）を使用  
Unityのゲーム用バージョン

### 準備

In [1]:
""" ライブラリ読み込み """
import torch
import numpy as np
import torch.optim as optim
from torch import nn
import os

In [2]:
""" GPU設定 """
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
'''モーションデータを扱うクラス '''
class motion_data():
    
    # numFrame:取り出すフレーム数、interval:取り出す間隔
    def __init__(self, numFrame):
        self.filelist = []       # 学習用データのファイルリスト
        self.dataset = []        # 学習用データセット（CSVから取得）
        self.frameTime = []      # 各フレームの時刻（CSVから取得）
        self.feature_len = 10     # 特徴量抽出後のベクトルの長さ（extract_feature関数に合わせて変更する）
        self.numFrame = numFrame
        
    # csvファイルから学習用データを取得
    def get_csv_data(self, dirName):
        self.dataset = []
        self.frameTime = []
        dirName = dirName + "/"
        self.filelist = os.listdir(dirName)
        for fileName in self.filelist:
            fname = dirName + fileName
            # 以下はnpyファイルを読み込むとき
            #newdata = np.load(fname)
            # 以下はcsvファイルを読み込むとき
            skiprows = 7    # skiprowsには先頭の飛ばす行数を指定
            csv_data = np.loadtxt(fname, delimiter=',', skiprows=skiprows, encoding="utf-8")
            newframeTime = csv_data[:, 1]
            newdata = np.array(csv_data[:, 2:])
            #newdata = newdata.reshape([csv_data.shape[0], csv_data.shape[1] // 3, 3])
            self.dataset.extend([newdata])
            self.frameTime.extend([newframeTime])
    
    # クォータニオンを回転軸と回転角度に分解（現在不使用）
    # rotationの中身は(x,y,z,w)のクォータニオン（npベクトル）
    # 数式の参照：http://gamemakerlearning.blog.fc2.com/blog-entry-205.html
    def toAngleAxis(self, rotation):
        angle = 2 * np.arccos(rotation[3])
        nx = rotation[0] / np.sin(angle / 2)
        ny = rotation[1] / np.sin(angle / 2)
        nz = rotation[2] / np.sin(angle / 2)
        axis = np.array([nx, ny, nz])
        return [angle, axis]
    
    def quaternion_product(self, q, p):
        Q = np.array([
            [ q[3], -q[2],  q[1],  q[0]],
            [ q[2],  q[3], -q[0],  q[1]],
            [-q[1],  q[0],  q[3],  q[2]],
            [-q[0], -q[1], -q[2],  q[3]]])
        qp = Q @ p
        return qp
    
    # 共役クォータニオンを返す
    def quaternion_inverse(self, q):
        inverse = np.array([-q[0], -q[1], -q[2], q[3]])
        return inverse
    
    # ベクトルをクォータニオンで回転
    # quaternionは(x,y,z,w)の4次元
    def quaternion_rotate_vector(self, vector, quaternion):
        v = np.array([vector[0], vector[1], vector[2], 0])
        q1 = quaternion.copy()    # 縦ベクトルにする
        q2 = self.quaternion_inverse(quaternion)
        q3 = self.quaternion_product(q1, v)
        ans = self.quaternion_product(q3, q2)
        return np.array([ans[0], ans[1], ans[2]])
    
    # 前額面・矢状面に投影したときの関節角度を返す
    # rotationの中身は(x,y,z,w)のクォータニオン（npベクトル）
    def frontal_sagittal_plane_angle(self, rotation):
        # (0,1,0)がrotationでどこへ移動するか
        # y軸回りの回転は検知できないが、前額面・矢状面ではどっちにしろx軸回りの回転を検知できないため問題ない
        v = self.quaternion_rotate_vector(np.array([0, 1, 0]), rotation)
        #print(v)
        frontal_angle = np.arctan2(v[0], v[1])
        sagittal_angle = np.arctan2(v[2], v[1])
        return frontal_angle, sagittal_angle
            
    # 特徴量抽出を行う関数
    # 圧縮後の次元数がself.feature_lenになるように注意
    # 引数：1時刻のデータ
    def extract_feature(self, data):
        # 必要なモーキャプデータのみ取り出す
        bone_data = data[0:91]    # boneのデータを抽出（上半身）
        vec7_data = bone_data.reshape([len(bone_data) // 7, 7])    # 座標(x,y,z)、回転(x,y,z,w)の7次元に変換
        rotation = vec7_data[:, 0:4]    # 座標データは使わない、回転(x,y,z,w)のみ
        # すべてのボーンは使用せず、一部のボーンだけ使用する
        chest = rotation[2]
        lu_arm = rotation[6]
        lf_arm = rotation[7]
        ru_arm = rotation[10]
        rf_arm = rotation[11]
        # 前額面での角度
        chest_fAng, chest_sAng = self.frontal_sagittal_plane_angle(chest)
        lu_arm_fAng, lu_arm_sAng = self.frontal_sagittal_plane_angle(lu_arm)
        lf_arm_fAng, lf_arm_sAng = self.frontal_sagittal_plane_angle(lf_arm)
        ru_arm_fAng, ru_arm_sAng = self.frontal_sagittal_plane_angle(ru_arm)
        rf_arm_fAng, rf_arm_sAng = self.frontal_sagittal_plane_angle(rf_arm)
        retData = [chest_fAng, lu_arm_fAng, lf_arm_fAng, ru_arm_fAng, rf_arm_fAng,
                   chest_sAng, lu_arm_sAng, lf_arm_sAng, ru_arm_sAng, rf_arm_sAng]
        return retData
    
    # モーションデータの中から引数intervalの間隔でデータを取り出し
    # それを複数パターンでndarray化して返す
    def split_interval_data(self, data, interval):
        numData = len(data) - (interval * (self.numFrame - 1))
        if (numData <= 0):
            return None
        retData = np.empty((numData, self.numFrame, self.feature_len))
        for i in range(numData):
            splitData = np.empty((self.numFrame, self.feature_len))    # 分割データの保存先
            for j in range(self.numFrame):
                splitData[j] = self.extract_feature(data[i + interval * j])
            retData[i] = splitData
        return retData

    # モーションデータをminIntervalから最大のインターバルで間引きしたnumFrameの長さのデータセットを返す
    # これにより様々な速さのモーションデータセットを得る
    def get_interval_data(self, minInterval, correctID, maxInterval=-1):
        numMotion = len(self.dataset)
        dataset = np.empty((0, self.numFrame, self.feature_len))
        for i in range(numMotion):    # すべてのモーションデータセットに対して
            if maxInterval < 0:
                maxInterval = (len(self.dataset[i]) - 1) // (self.numFrame - 1)
            for interval in range(minInterval, maxInterval + 1):
                splitdata = self.split_interval_data(self.dataset[i], interval)
                if splitdata is not None:
                    dataset = np.concatenate([dataset, splitdata])
        correct_data = np.ones(len(dataset), dtype=np.int64) * correctID
        return [dataset, correct_data]

In [4]:
# モーションデータのオブジェクト
numFrame = 12     # モーションデータのフレーム数
interval = 2      # モーションデータを何フレームに一度取り出すか
maxInterval = 4
motion_count = 4
m = []
for i in range(motion_count):
    m.append(motion_data(numFrame))     # モーションのデータオブジェクト
    
#print(m[0].frontal_sagittal_plane_angle(np.array([-0.14832, -0.35363, 0.35720, 0.85168])))
#print(m[0].frontal_sagittal_plane_angle(np.array([0.00000, 0.00000, 0.38268, 0.92388])))
#print(m[0].frontal_sagittal_plane_angle(np.array([-0.027334,0.023318,-0.027962,0.998963])))

In [5]:
""" LSTMのモデルクラス """
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.feature_size = 10          # 特徴量x(t)の次元
        self.hidden_layer_size = 30    # 隠れ層のサイズ
        self.lstm_layers = 1           # LSTMのレイヤー数　(LSTMを何層重ねるか)
        self.output_size = 4           # 出力層のサイズ
        
        self.lstm = torch.nn.LSTM(self.feature_size, 
                                  self.hidden_layer_size, 
                                  num_layers = self.lstm_layers)
        
        self.fc = torch.nn.Linear(self.hidden_layer_size, self.output_size)
        
    def init_hidden_cell(self, batch_size): # LSTMの隠れ層 hidden と記憶セル cell を初期化
        hedden = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size)
        cell = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size)        
        return (hedden, cell)

    def forward(self, x):
        batch_size = x.shape[0]
        self.hidden_cell = self.init_hidden_cell(batch_size)
        x = x.permute(1, 0, 2)                                   # (Batch, Seqence, Feature) -> (Seqence , Batch, Feature)
        
        lstm_out, (h_n, c_n) = self.lstm(x, self.hidden_cell)    # LSTMの入力データのShapeは(Seqence, Batch, Feature)
                                                                 # (h_n) のShapeは (num_layers, batch, hidden_size)
        x = h_n[-1,:,:]                                          # lstm_layersの最後のレイヤーを取り出す  (B, h)
                                                                 # lstm_outの最後尾と同じ？
        x = self.fc(x)
        
        return x

### 学習

In [6]:
import random

""" データの準備 """
# 訓練データ（とテストデータ）から一度に何個のデータを読み込むかを指定する値
BATCH_SIZE = 20

# モーションデータを取り出して分割
m[0].get_csv_data("./stop")
m[1].get_csv_data("./walk")
m[2].get_csv_data("./jump")
m[3].get_csv_data("./run")

input_data = []
correct_data = []
data_size = []
for i in range(motion_count):
    print("Motion:%d Processing data..." % i)
    ms, mc = m[i].get_interval_data(interval, i, maxInterval)
    input_data.append(ms)
    correct_data.append(mc)
    data_size.append(len(mc))
print("Data size: " + str(data_size))
    
# 学習データの数をモーションごとにそろえる（少ないデータの数に合わせる）
min_data_size = min(data_size)
for i in range(motion_count):
    idx = random.sample(list(range(data_size[i])), k=min_data_size)
    input_data[i] = (input_data[i])[idx]
    correct_data[i] = (correct_data[i])[idx]

# 学習用モーションデータと正解ラベルのデータを作成
input_data = np.concatenate(input_data)    # リストの要素同士を結合
correct_data = np.concatenate(correct_data)

np.save('input_data', input_data)
np.save('correct_data', correct_data)
print(input_data * 180 / np.pi)

Motion:0 Processing data...
Motion:1 Processing data...
Motion:2 Processing data...
Motion:3 Processing data...
Data size: [11976, 17169, 17268, 17142]
[[[  -1.92512146   82.8526687    68.05346059 ...   -2.44649434
     63.76711057    2.10356247]
  [  -1.85287886   82.90238844   68.10230082 ...   -2.80240763
     64.44451718    2.62035988]
  [  -1.78624262   82.92967758   68.12284921 ...   -3.25403922
     65.10485542    3.26375069]
  ...
  [  -1.45192054   82.96741389   67.9591821  ...   -5.31770533
     68.37840188    5.91648767]
  [  -1.44793628   82.96583961   67.92801253 ...   -5.36597265
     68.49885688    6.21753117]
  [  -1.44587957   82.98921291   67.93024777 ...   -5.57193709
     68.54969399    6.42759582]]

 [[   2.98480125   67.66486709   72.62274461 ...  -73.91467744
     71.08403131   68.95009664]
  [   2.93412174   67.93326828   72.57392737 ...  -73.65363645
     70.95063362   68.85222352]
  [   2.87692526   68.29368515   72.5499849  ...  -73.37558229
     70.78919264 

In [7]:
input_data = np.load('input_data.npy')
correct_data = np.load('correct_data.npy')

# テンソルに変換
input_data = torch.FloatTensor(input_data)
correct_data = torch.LongTensor(correct_data)
#input_data = torch.cuda.FloatTensor(input_data)
#correct_data = torch.cuda.LongTensor(correct_data)

# データセットの準備
dataset = torch.utils.data.TensorDataset(input_data, correct_data)
# 学習用データと検証用データに分割
train_size = int(0.8 * len(dataset))    # 学習用データのサイズ（全体の8割）
test_size = len(dataset) - train_size    # 検証用データのサイズ
trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
#print(vars(dataset))

In [8]:
print(input_data[20] * 180 / np.pi)

tensor([[ -1.9983,  74.0365,  73.0017,  67.3340,  68.7269,  -2.0990, -78.6486,
         -67.2468,  70.8273,  67.2607],
        [ -2.0323,  73.9830,  72.9460,  67.2954,  68.6934,  -2.0973, -78.6513,
         -67.2347,  70.9136,  67.3599],
        [ -2.0685,  73.9326,  72.8851,  67.3235,  68.7239,  -2.0924, -78.6676,
         -67.2430,  70.9917,  67.4230],
        [ -2.1008,  73.9044,  72.8339,  67.3545,  68.7579,  -2.0812, -78.6973,
         -67.2719,  71.1048,  67.5247],
        [ -2.1243,  73.9198,  72.8055,  67.3720,  68.7789,  -2.0678, -78.7435,
         -67.3019,  71.2491,  67.6932],
        [ -2.1548,  73.9699,  72.8021,  67.4538,  68.8427,  -2.0655, -78.8005,
         -67.3482,  71.3610,  67.8393],
        [ -2.1825,  74.0223,  72.7934,  67.6226,  68.9585,  -2.0670, -78.8512,
         -67.3878,  71.4321,  67.9509],
        [ -2.2099,  74.0570,  72.7692,  67.7168,  69.0576,  -2.0874, -78.9008,
         -67.4083,  71.5243,  68.0395],
        [ -2.2417,  74.0884,  72.7458,  67.8457,

In [9]:
""" 設定 """
# モデル作成
net = Net().to(device)
# 損失関数（計算結果と正解ラベルの誤差を比較、それを基に最適化）
criterion = torch.nn.CrossEntropyLoss()  # CrossEntropyLossは損失関数に多クラス分類でよく使われる
# 最適化アルゴリズム
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  # SGD（確率的勾配降下法）
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [10]:
""" 学習 """
EPOCHS = 1  # すべての入力に対してn回実行

for epoch in range(1, EPOCHS + 1):
    running_loss = 0.0  # 平均値出力用
    for count, item in enumerate(trainloader, 1):  # BATCH_SIZEごとに実行するため、このときcountの値を増やす
        inputs, labels = item  # trainloader経由でデータを20個取り出す
        
        # CUDAで使えるようキャスト
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # 重みとバイアスの更新で内部的に使用するデータセット
        
        # Runs the forward pass with autocasting.
        outputs = net(inputs)  # ニューラルネットワークにデータを入力
        
        loss = criterion(outputs, labels)  # 正解ラベルとの比較

        loss.backward()  # 誤差逆伝播
        optimizer.step()  # 重みとバイアスの更新

        running_loss += loss.item()
        times = 100
        if count % times == 0:
            print(f'#{epoch}, data: {count * BATCH_SIZE}, running_loss: {running_loss / times:1.3f}')
            running_loss = 0.0

torch.save(net, 'body1.pth')
print('Training Finished')

#1, data: 2000, running_loss: 1.076
#1, data: 4000, running_loss: 0.259
#1, data: 6000, running_loss: 0.149
#1, data: 8000, running_loss: 0.118
#1, data: 10000, running_loss: 0.081
#1, data: 12000, running_loss: 0.146
#1, data: 14000, running_loss: 0.078
#1, data: 16000, running_loss: 0.089
#1, data: 18000, running_loss: 0.063
#1, data: 20000, running_loss: 0.043
#1, data: 22000, running_loss: 0.048
#1, data: 24000, running_loss: 0.050
#1, data: 26000, running_loss: 0.057
#1, data: 28000, running_loss: 0.056
#1, data: 30000, running_loss: 0.029
#1, data: 32000, running_loss: 0.044
#1, data: 34000, running_loss: 0.029
#1, data: 36000, running_loss: 0.045
#1, data: 38000, running_loss: 0.035
Training Finished


In [11]:
""" 結果出力 """
_, predicted = torch.max(outputs, 1)
print(predicted)
print(labels)

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += len(outputs)
        correct += (predicted == labels).sum().item()

print(f'correct: {correct}, accuracy: {correct} / {total} = {100 * correct / total} %')


tensor([1, 0, 3])
tensor([1, 0, 3])
correct: 9429, accuracy: 9429 / 9581 = 98.41352677173573 %


### UDPの準備

In [12]:
from socket import *

class udp_client:
    def __init__(self, SrcIP="127.0.0.1", SrcPort=11111, DstIP="127.0.0.1", DstPort=22222):
        # 送信側アドレスをtupleに格納
        self.SrcAddr = (SrcIP, SrcPort)
        # 受信側アドレスをtupleに格納
        self.DstAddr = (DstIP, DstPort)
        
        self.BUFSIZE = 1024 

        # ソケット作成
        self.udpClntSock = socket(AF_INET, SOCK_DGRAM)
        # 送信側アドレスでソケットを設定
        self.udpClntSock.bind(self.SrcAddr)
        
    
    def __delete__(self):
        del self.udpClntSock
        
        
    def send(self, data):
        # バイナリに変換
        data = data.encode('utf-8')

        # 受信側アドレスに送信
        self.udpClntSock.sendto(data, self.DstAddr)
        
        
    def receive(self):
        data, addr = self.udpClntSock.recvfrom(self.BUFSIZE)
        data = data.decode()
        return data

### リアルタイム認識

In [ ]:
from IPython.display import clear_output
import torch
import cv2
import mediapipe as mp
import numpy as np

import time

""" GPU設定 """
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

""" 識別 """
net = torch.load('body1.pth')

# Unityへ送るUDP
udp = udp_client()

test_motion = motion_data(numFrame)
test_motion.get_csv_data("./test_motion2")
data, _ = test_motion.get_interval_data(interval, 0, interval)


start_time = time.time()
for index in range(0, len(data), interval):
    cv2.imshow("Window", np.zeros([256, 256]))
    
    key = cv2.waitKey(1) & 0xFF
    if key == 27:    # Escで終了
        break
    
    # intervalの間最も頻出だったモーションを1位とする
    answers = []
    for i in range(0, interval):
        if index + i >= len(data):
            break
        motion = np.expand_dims(data[index+i], 0)
        motion = torch.FloatTensor(motion)
        outputs = net(motion)  # ニューラルネットワークにデータを入力
        value, predicted_idx = torch.max(outputs, 1)
        answer = int(predicted_idx[0])
        answers.append(answer)
    
    # 最頻値を求める
    uniqs, counts = np.unique(answers, return_counts=True)
    answer = (uniqs[counts == np.amax(counts)])[0]
    clear_output(wait=True)
    if (answer == 0):
        print("stop")
        udp.send("stop")
    elif (answer == 1):
        print("walk")
        udp.send("walk")
    elif (answer == 2):
        print("jump")
        udp.send("jump")
    elif (answer == 3):
        print("run")
        udp.send("run")
    else:
        print("error")
    while (time.time() - start_time < (index + 1) / 120):
        time.sleep(0.001)
        

print("finish")
print(time.time() - start_time)
cv2.destroyAllWindows()
del udp

walk


In [24]:
cv2.destroyAllWindows()
del udp